In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: C:\Users\lppap\Documents\master\scalable_ML\mlfs-book
HopsworksSettings initialized!


## Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion



### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

In [13]:

project = hopsworks.login(engine="python")
fs = project.get_feature_store()
secrets = hopsworks.get_secrets_api()

AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
# The name of the Hopsworks secret (e.g., SENSOR_LOCATION_JSON_ecke-taborstrasse-glockengasse)
# is provided as an environment variable when the notebook runs.
# If no environment variable is passed (e.g., when running locally),
# it defaults to "SENSOR_LOCATION_JSON".
# This allows the same notebook to dynamically load the configuration
# for any air quality station based on the runtime parameter.

secret_name = os.getenv("SENSOR_LOCATION_JSON", "SENSOR_LOCATION_JSON")
print(f" Using secret name: {secret_name}")

secret = secrets.get_secret(secret_name).value
print(f" Using secret: {secret}")

station_cfg = json.loads(secret)

# Extract parameters
STATION_UID = station_cfg["station_uid"]
COUNTRY = station_cfg["country"]
CITY = station_cfg["city"]
STREET = station_cfg["street"]
URL = station_cfg["url"]
LAT = float(station_cfg["latitude"])
LON = float(station_cfg["longitude"])

today = datetime.date.today()

print(f"🌍 {CITY}, {COUNTRY}")
print(f"📍 {STREET} [UID={STATION_UID}]")
print(f"🔗 {URL}")
print(f"📡 Coordinates: {LAT}, {LON}")


##

2025-11-13 10:23:42,130 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-13 10:23:42,139 INFO: Initializing external client
2025-11-13 10:23:42,142 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-13 10:23:43,728 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279154
🌍 Vienna, Austria
📍 Ecke TaborstraÃŸe - Glockengasse [UID=2860]
🔗 https://aqicn.org/station/@2860/
📡 Coordinates: 48.216739, 16.380918


### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [11]:
# Retrieve feature groups
stations_air_quality_daily = fs.get_feature_group(
    name='stations_air_quality_daily',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API + create t1, t2 and t3</span>


In [24]:
import requests
import pandas as pd

feed_url = f"https://api.waqi.info/feed/@{STATION_UID}/"

'''
aq_today_df = util.get_pm25(
    aqicn_url=feed_url,
    country=COUNTRY,
    city=CITY,
    street=STREET,
    day=today,
    AQI_API_KEY=AQICN_API_KEY
)
'''

base_day = pd.to_datetime(today).normalize()
dfs = []

# 0 = today, 1 = yesterday, 2 = -2 days, 3 = -3 days
for delta in range(0, 4):
    day = base_day - pd.Timedelta(days=delta)
    df_day = util.get_pm25(
        aqicn_url=feed_url,
        country=COUNTRY,
        city=CITY,
        street=STREET,
        day=day,
        AQI_API_KEY=AQICN_API_KEY,
    )
    
    dfs.append(df_day)
    
# 4-day history for this station
aq_hist_df = pd.concat(dfs, ignore_index=True)

# if it's not already in the DF, add station_uid
aq_hist_df["station_uid"] = STATION_UID

aq_hist_df = aq_hist_df.sort_values(["station_uid", "date"])
aq_hist_df["pm25_t1"] = aq_hist_df.groupby("station_uid")["pm25"].shift(1)
aq_hist_df["pm25_t2"] = aq_hist_df.groupby("station_uid")["pm25"].shift(2)
aq_hist_df["pm25_t3"] = aq_hist_df.groupby("station_uid")["pm25"].shift(3)

# keep only the base day (today) and the rows where all 3 lag values exist
rows_to_insert = aq_hist_df[
    (aq_hist_df["date"].dt.normalize() == base_day)
].dropna(subset=["pm25_t1", "pm25_t2", "pm25_t3"])


0    pm25  country    city                            street       date  \
0  34.0  Austria  Vienna  Ecke TaborstraÃŸe - Glockengasse 2025-11-13   

                                 url  
0  https://api.waqi.info/feed/@2860/  
1    pm25  country    city                            street       date  \
0  34.0  Austria  Vienna  Ecke TaborstraÃŸe - Glockengasse 2025-11-12   

                                 url  
0  https://api.waqi.info/feed/@2860/  
2    pm25  country    city                            street       date  \
0  34.0  Austria  Vienna  Ecke TaborstraÃŸe - Glockengasse 2025-11-11   

                                 url  
0  https://api.waqi.info/feed/@2860/  
3    pm25  country    city                            street       date  \
0  34.0  Austria  Vienna  Ecke TaborstraÃŸe - Glockengasse 2025-11-10   

                                 url  
0  https://api.waqi.info/feed/@2860/  
   pm25  country    city                            street       date  \
0  34.0  Austria  V

'\nstations_air_quality_daily.insert(\n    rows_to_insert,\n    operation="upsert",\n    write_options={"wait_for_job": True},\n)\n'

## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [ ]:
hourly_df = util.get_hourly_weather_forecast(CITY, LAT, LON)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = CITY
daily_df

In [ ]:
daily_df.info()

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [14]:
# Insert new data
stations_air_quality_daily.insert(
    rows_to_insert,
    operation="upsert",
    write_options={"wait_for_job": True},
)

Uploading Dataframe: 100.00% |█████████████████████████████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: stations_air_quality_daily_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279154/jobs/named/stations_air_quality_daily_1_offline_fg_materialization/executions


(Job('stations_air_quality_daily_1_offline_fg_materialization', 'SPARK'), None)

In [ ]:
# Insert new data
weather_fg.insert(daily_df, wait=True)